Unfortunately Github doesn't render ipywidgets, which allow us to interactively alter the parameters of the model. If you want to mess with the parameters yourself then download this file, and also install and activate ipywidgets, which you can do in the command line interface with the following commands:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

*things I am still confused about*
<br>-calculating kinetic parameters such as $k^+_R$, $k^+_o$ $k^+_c$ and $\tilde{k_2}$
<br>-calculating $j_{pga,net}$ as a function of $j_{oA}$ and $j_{cA}$

<img src="SUschema.png">
Fig 1: Schematic to represent the various reactions of photosynthesis through synthesising units. Each synthesizing unit has incoming and outgoing fluxes. $j_{ph}$ is the electron transport flux, which fuels production of ATP and NADPH which are used in the reduction of PGA following both oxygenation and carboxylation. $j_{cA}$ and $j_{oA}$ are the Carbon and Oxygen assimilation fluxes, respectively, and $j_{prod}$ is the outgoing sucrose synthesis flux. $j_R$ is the RuBP regeneration flux, $j_{pgaO}$ is the PGA synthesis flux through Carbon recovery after oxygenation, and $j_{pga,net}$ is the net PGA synthesis flux.

<img src = "carbo_oxo.png">
Fig 2: Schematic for simultaneous oxygenation and carboxylation of RuBP according to SU kinetics, whereby forward reactions are much stronger than reverse reactions. 

Based on the schematic in figure 1, the net rate of photosynthesis, $j_{prod}$ incorporates the electron transport flux $j_{ph}$ and the PGA synthesis flux $j_{pga,net}$ in parallel. Formulated according to Kooijman 1998's Synthesizing Unit kinetics:
\begin{equation}
    j_{prod} = (\frac{1}{k_E} + \frac{1}{j_{ph}} + \frac{1}{j_{pga,net}} - \frac{1}{j_{ph} + j_{pga,net}})^{-1}
\end{equation}
where $k_E$ represents the maximum velocity of enzymes in the reduction of PGA.
$j_{pga,net} = 2j_{cA} + \frac{3}{2}j_{oA}$ as for every Carbon consumed, 2 PGA are produced, and for every 2 Oxygen consumed, 3 PGA are produced.

We can derive $j_{cA}$ as $-d[C]_T/dt$ based on figure 2, which yields 
$$j_{cA} = -k^+_c[C][E]_T(1 + \frac{k^+_R[R]}{k^+_o[O] + k^+_c[C]})f_E $$
where 
$$ f_E = \frac{[E]}{[E]_T} = (1 + (\frac{1}{k^+_R[R]} + 1)(\tilde{k^+_o}[O] +  \tilde{k^+_c}[C]) + k^+_R[R]\frac{1 + \tilde{k^+_o}[O] +  \tilde{k^+_c}[C]}{k^+_o[O] +  k^+_c[C]})^{-1} $$
where $\tilde{k^+_o} = k^+_o/\tilde{k_{ERO}}$ and $\tilde{k^+_c} = k^+_c/\tilde{k_{ERC}}$, whereby $\tilde{k_{ERC}} = k_{ERC}+k^-_c+k^-_R$ and $\tilde{k_{ERO}} = k_{ERO}+k^-_o+k^-_R$
<br>
By symmetry,
$$j_{oA} = -k^+_o[O][E]_T(1 + \frac{k^+_R[R]}{k^+_c[C] + k^+_o[O]})f_E $$

We use the electron transport rate derived by Ye et al (Ye 2013):
$$ j_{ph} = \alpha_e \frac{1-\beta_eI}{1+\gamma_eI}I
$$
where $\alpha_e$ is the initial slope of the $j_{ph}$-$I$ curve, and $\beta_e$ and $\gamma_e$ are the coefficiencts for photoinhibition and light saturation, respectively. $I$ is irradiance.

In [1]:
#dependencies
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [36]:
#Parameter values

#from Farquhar 1980
#turnover rate for carboxylation
kc = 2.5
#turnover rate for oxygenation
ko = .21*kc
#light intensity
I = 150
#total enzyme concentration
Et = 87.2
#CO2 concentration
C = 230
#O2 concentration
O = 210

#from Ye 2020
alpha = .295
beta = 2.42*(10**(-3))
gamma = 1.26*(10**(-4))

#couldnt find turnover # for RuBP so am setting kr, k2 = kc
kr = kc
kprod = kc
kE = kc

#also guesses here
kERC = .9*kc
kERO = .9*ko

In [3]:
#model for jprod
def jprod(R, C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    return jprod

In [4]:
#plotting jprod against RuBP
@interact(Rsteps= (0, 5000, 10))
def simulateR(C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Rsteps=300):
    vcr = []
    for k in range(1,Rsteps):
        vcr.append(jprod(k, C=C, O=O, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    plt.plot(range(1,Rsteps), vcr)
    plt.xlabel('RuBP')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=230, description='C', max=690, min=-230), IntSlider(value=210, descripti…

Plotting jcA by itself

In [5]:
@interact(Rsteps=(0,10000,10))
def simulateC(C = 100, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Rsteps=300):
    jcAlist=[]
    for R in range(1,Rsteps):
        ksum = ko*O/kERO + kc*C/kERO
        fE = (1 + ((1/(kr*R))+1)*ksum + kr*(1+ksum)/(ko*O + kc*C))**(-1)
        jcA = kc*C*Et*(1+(kr*R/ksum))*fE
        jcAlist.append(jcA)
    plt.plot(range(1,Rsteps), jcAlist)
    plt.xlabel('RuBP')
    plt.ylabel('JcA')
    plt.show()

interactive(children=(IntSlider(value=100, description='C', max=300, min=-100), IntSlider(value=210, descripti…

In [6]:
#plotting jprod against irradiance
@interact(isteps= (0, 5000, 100))
def simulateI(R=300, C = 230, O =210, kc = kc, ko = ko, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, isteps=300):
    vci = []
    for i in range(1,isteps):
        vci.append(jprod(R=R, C=C, O=O, kc = kc, ko = ko, I = i, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    plt.plot(range(1,isteps), vci)
    plt.xlabel('Irradiance')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=300, description='R', max=900, min=-300), IntSlider(value=230, descripti…

*as I increases jprod saturates and then has very large spikes*

<h3> Update 2/10</h3>
Since the last update to this notebook, I've added 1) two alternative formulations for j_ph, one where flourescence depends on light and another that frames the light reactions as a circuit, 2) A model for RuBP regeneration, which requires further analysis, and 3) an integrated model for the light reactions that seeks to balance incoming and outgoing fluxes

<h3>Alternative models for $j_{ph}$</h3>
Ye's model for $j_{ph}$ faces two main issues. The first is that $j_{ph}$ can become negative, and the second is that the occupation probabilities for different reaction pathways do not depend on light. Photoinhibitory regulatory mechanisms have a complex relationship with light. For dark acclimated leaves exposed to light, non-photochemical quenching (NPQ) also known as thermal dissipation, rapidly increases and then decreases to a steady state. NPQ competes with flourescence and photochemical quenching, the pathway that contributes to photosynthesis.

<h4>Model A: Incorporating light dependant flourescence</h4>
We can potentially correct the Ye model by revising the original derivation and multiplying $\xi_3R_{ki}$ by $I$, leading to:
$$
j_{ph} = \alpha_e \frac{1-\frac{\beta_eI}{z_1I+z_2}}{1+\gamma_eI}I
$$

In [7]:
zun = .21
zdo = .33

In [8]:
def jpha(I, zun = zun, zdo = zdo):
    frac = beta*I/(zun*I+zdo)
    jph = I*alpha*(1-frac)/(1+gamma*I)
    return jph

In [9]:
#model Aa: multiplying I by both parameters
def jphaa(I, zun=zun, zdo=zdo):
    frac = beta*I/(zun*I+zdo*I)
    jph = I*alpha*(1-frac)/(1+gamma*I)
    return jph

<h4>Circuit formulation</h4>
We can treat both light dependant ($R_I$) and light independant ($R_C$) photoinhibitory mechanisms as resistances to photochemistry, where $R_I = R_{I0}I$. $I$ is analogous to voltage. 
<img src = "circuit.png">
Thus, using Ohm's law, ${j_ph} = \frac{I}{R_{I0}I+R_C}$. At high light intensities, $j_{ph}$ saturates to $\frac{1}{R_{I0}}$ so we can say $R_{I0} = 1/j_{max}$

In [10]:
Imax = 700
Rc = .01

In [11]:
def jphb(I, Imax=Imax, Rc=Rc):
    jph = I/((I/Imax)+Rc)
    return jph

In [12]:
@interact()
def simulatejph(irange=100000, zun=zun, zdo=zdo, Imax = Imax, Rc=Rc):
    jphalist = []
    jphaalist = []
    jphblist = []
    for i in range(1,irange):
        jphalist.append(jpha(i, zun, zdo))
        jphaalist.append(jphaa(i, zun, zdo))
        jphblist.append(jphb(i, Imax, Rc))
    plt.plot(range(1,irange), jphalist, label='model A')
    plt.plot(range(1, irange), jphalist, label = 'model Aa')
    plt.plot(range(1,irange), jphblist, label = 'model B')
    plt.xlabel('Irradiance')
    plt.ylabel('jph')
    plt.legend()
    plt.show()
    

interactive(children=(IntSlider(value=100000, description='irange', max=300000, min=-100000), FloatSlider(valu…

*Not much of a difference between model A and model Aa*

<h4>Incorporating RuBP regeneration</h4>
Based on figures 1 and 2, we can derive a dynamical system for the consumption and regeneration of $R$:
$$
\frac{d[R]}{dt} = -k^+_R[R]([E] + [EC] + [EO]) + \frac{5}{6}j_{prod} = -[E]_Tf_E(k^+_R[R] + k^+_c[C] + k^+_o[O]) + \frac{5}{6}j_{prod}
$$
which reflects that 5/6 of PGA produced by carboxylation is used for RuBP regeneration

In [13]:
def dRdt(R, C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + (5/6)*jprod
    return dRdt

In [22]:
@interact(alpha=(0,10,1), dt = (0,.01, .001), tsteps=(0,100000, 10))
def simulateR(dt = .001, tsteps = 1000, Ro = 100, C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    Rlist = np.zeros(tsteps)
    Rlist[0] = Ro
    jprodlist = np.zeros(tsteps)
    jprodlist[0] = jprod(Ro)
    for t in range(tsteps-1):
        jprodlist[t+1] = jprod(Rlist[t])
        Rlist[t+1] = Rlist[t] + dRdt(Rlist[t])*dt
    plt.plot(range(tsteps), Rlist, label='RuBp')
    plt.plot(range(tsteps), jprodlist, label = 'jprod')
    plt.legend()
    plt.xlabel('time')
    plt.show()
    #print(Rlist)

interactive(children=(FloatSlider(value=0.001, description='dt', max=0.01, step=0.001), IntSlider(value=1000, …

In [25]:
#stability analysis for dRdt
from sympy import solveset, symbols, S

In [ ]:
R, C, O, kc, ko, I, Et, alpha, beta, gamma, kr, kE, kERO, kERC = symbols("R, C, O, kc, ko, I, Et, alpha, beta, gamma, kr, kE, kERO, kERC")
jph = I*alpha*(1-beta*I)/(1+gamma*I)
ksum = ko*O/kERO + kc*C/kERC
fE = (1 + ((1/(kr*R))+1)*ksum + kr*(1+ksum)/(ko*O + kc*C))**(-1)
jcA = kc*C*Et*(1+(kr*R/ksum))*fE
joA = ko*O*Et*(1+(kr*R/ksum))*fE
jpga = (2*jcA+1.5*joA)
jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + (5/6)*jprod
print(solveset(dRdt, R))

<h4>Next steps for RuBP model</h4>
As we can see if we run the above code block, the stability analysis times out, so we will have to look for other ways to analyze the RuBP model.
<br>
One possible step would be to non-dimensionalize the model, which would reduce the number of parameters.
<br>
We can also conduct a Bayesian optimization to explore the parameter space, and qualitatively infer the parameter ranges that lead to different behaviors.

<h3> Modelling light reactions by balancing fluxes </h3>
<img src = "lightreaction.png">
We seek to mechanistically model the light reactions by balancing incoming and outgoing fluxes. Light ($j_I$) is intercepted by flourescence ($j_F$) and thermal dissipation ($j_{NPQ}$), the latter of which is controlled by the pH gradient, which is determined by the difference between the incoming flux from photochemical quenching, $j_{PQ}(t-1)$, and outgoing ATP flux, $j_{ATP}(t-1)$. $j_{ATP}$ depends on the rate of the Calvin Cycle, which in turn depends on $j_{PQ}$. Thus, we can formulate $j_{PQ}$ as a discrete dynamical system, where $j_{PQ}(t) = f(j_{PQ}(t-1))$.
Ultimately, it would be interesting if the system is able to imitate the following experimental results:
<img src="darkacc.png">

As we can see in the schematic, $J_{PQ}$ is the portion of irradiance not used for flourescence and thermal dissipation.
$$
j_{PQ}(t) = j_I - j_F - j_{NPQ}(t)
$$
Where we can think of $j_{NPQ}$ as consuming a fraction $f_{NPQ}$ of $j_H$, which represents the build up of protons due to the difference between the incoming $j_{PQ}(t-1)$ and outgoing $j_{ATP}(t-1)$.
$$
j_{NPQ} = f_{NPQ}(j_{PQ}(t-1) - j_{ATP}(t-1))
$$
We treat ATP synthase as a synthesizing unit, where light and ADP from the Calvin cycle are consumed to form ATP:
$$
j_{ATP}(t-1) = (\frac{1}{j_{syn}} + \frac{1}{j_{PQ}(t-1)} + \frac{1}{j_{ADP}(t-1)} - \frac{1}{j_{PQ}(t-1) + j_{ADP}(t-1)})^{-1}
$$
Where 
$$
j_{ADP}(t-1) = \frac{2}{3}j_{prod}(t-1) + \frac{1}{3}j_{RuBP}(t-1)
$$
In $j_{prod}$ and $j_{RuBP}$, $j_{ph}$ would be replaced with $j_{PQ}(t-1)$

In [37]:
def jprodalt(pq, I, R):
    jph = pq
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + (5/6)*jprod
    return jprod, dRdt

In [44]:
def jpq(pq, I, R, jf=.1*I, fnpq=.2, jsyn=1.5):
    jp, jr = jprodalt(pq, I, R)
    jadp = 2*jp/3 + jr/3
    jatp = (1/jsyn + 1/pq + 1/jadp - 1/(pq + jadp))**(-1)
    jnpq = fnpq*(pq - jatp)
    jpq = I - jf - jnpq
    return jpq, jr, jnpq

In [45]:
def logistic(Io, Imax, x, kl):
    f = Imax/(1+np.exp(-kl*(x-Io)))
    return f

In [50]:
@interact(pqo = (0,10,.1), Ro=(0,500,10), time = (0,1000,1))
def integrated(pqo, Ro, Io=100, Imax=500, kl = 1, time=1000, dt = .001, C = 230, O =210, kc = kc, ko = ko, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    tsteps = np.linspace(0, time, time*10)
    jpqlist = np.zeros(len(tsteps))
    Rlist = np.zeros(len(tsteps))
    npqlist = np.zeros(len(tsteps))
    jpqlist[0] = pqo
    Rlist[0] = Ro
    ilist = [logistic(Io, Imax, t, kl) for t in tsteps]
    for t in range(len(tsteps)-1):
        jp, jr, npq = jpq(jpqlist[t], ilist[t], Rlist[t])
        Rlist[t+1] = Rlist[t] + jr*dt
        jpqlist[t+1] = jp
        npqlist[t] = npq
    plt.plot(tsteps, jpqlist, label='Jpq')
    #plt.plot(tsteps, Rlist, label = 'RuBP')
    plt.plot(tsteps, ilist, label='Irradiance')
    npqlist[-1] = npqlist[-2]
    plt.plot(tsteps, npqlist, label='NPQ')
    plt.legend()
    plt.xlabel('time')
    plt.show()
        

interactive(children=(FloatSlider(value=5.0, description='pqo', max=10.0), IntSlider(value=250, description='R…